# Update Certificates  
SSLCertVerificationError


In [9]:
! pip3 install certifi


In [10]:
import os
import certifi

os.environ['SSL_CERT_FILE'] = certifi.where()


# Downlaoding the Dataset 

In [11]:
from convokit import Corpus, download
corpus = Corpus(filename=download("movie-corpus"))

No configuration file found at /Users/wajeeh/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
default_backend: mem


In [12]:
corpus.print_summary_stats()

Number of Speakers: 9035
Number of Utterances: 304713
Number of Conversations: 83097


In [28]:
# Print the first 10 utterances
for utterance in list(corpus.iter_utterances())[:10]:
    print(utterance.text)


They do not!
They do to!
I hope so.
She okay?
Let's go.
Wow
Okay -- you're gonna need to learn how to lie.
No
I'm kidding.  You know how sometimes you just become this "persona"?  And you don't know how to quit?
Like my fear of wearing pastels?


In [31]:
# Print the first 10 speakers
for speaker in list(corpus.iter_speakers())[:10]:
    print(speaker.meta)


ConvoKitMeta({'character_name': 'BIANCA', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'f', 'credit_pos': '4'})
ConvoKitMeta({'character_name': 'CAMERON', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'm', 'credit_pos': '3'})
ConvoKitMeta({'character_name': 'CHASTITY', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': '?', 'credit_pos': '?'})
ConvoKitMeta({'character_name': 'JOEY', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'm', 'credit_pos': '6'})
ConvoKitMeta({'character_name': 'KAT', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'f', 'credit_pos': '2'})
ConvoKitMeta({'character_name': 'WALTER', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'm', 'credit_pos': '9'})
ConvoKitMeta({'character_name': 'BRUCE', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': '?', 'credit_pos': '?'})
ConvoKitMeta({'characte

In [32]:
# Print the first 5 conversations
for convo in list(corpus.iter_conversations())[:5]:
    print(convo.get_utterance_ids())


['L1045', 'L1044']
['L985', 'L984']
['L925', 'L924']
['L872', 'L871', 'L870']
['L869', 'L868', 'L867', 'L866']


# Preparing Data for Bert

In [36]:
texts = [utterance.text for utterance in corpus.iter_utterances()]


# Load BERT for NER

In [37]:
from transformers import BertTokenizer, BertForTokenClassification
from torch.nn.functional import softmax

tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
model = BertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

# Perform NER with BERT

In [45]:
def bert_ner(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs).logits
    predictions = softmax(outputs, dim=2)
    entities = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids'].tolist()]

    return [(word, model.config.id2label[pred.argmax().item()]) for word, pred in zip(entities[0], predictions[0])]

# Example usage
for text in texts[:10]:  # Analyze first 10 texts
    print("Text:", text)
    print("Entities:", bert_ner(text, tokenizer, model))

    


Text: They do not!
Entities: [('[CLS]', 'O'), ('They', 'O'), ('do', 'O'), ('not', 'O'), ('!', 'O'), ('[SEP]', 'O')]
Text: They do to!
Entities: [('[CLS]', 'O'), ('They', 'O'), ('do', 'O'), ('to', 'O'), ('!', 'O'), ('[SEP]', 'O')]
Text: I hope so.
Entities: [('[CLS]', 'O'), ('I', 'O'), ('hope', 'O'), ('so', 'O'), ('.', 'O'), ('[SEP]', 'O')]
Text: She okay?
Entities: [('[CLS]', 'O'), ('She', 'O'), ('okay', 'O'), ('?', 'O'), ('[SEP]', 'O')]
Text: Let's go.
Entities: [('[CLS]', 'O'), ('Let', 'O'), ("'", 'O'), ('s', 'O'), ('go', 'O'), ('.', 'O'), ('[SEP]', 'O')]
Text: Wow
Entities: [('[CLS]', 'O'), ('Wow', 'O'), ('[SEP]', 'O')]
Text: Okay -- you're gonna need to learn how to lie.
Entities: [('[CLS]', 'O'), ('Okay', 'O'), ('-', 'O'), ('-', 'O'), ('you', 'O'), ("'", 'O'), ('re', 'O'), ('gonna', 'O'), ('need', 'O'), ('to', 'O'), ('learn', 'O'), ('how', 'O'), ('to', 'O'), ('lie', 'O'), ('.', 'O'), ('[SEP]', 'O')]
Text: No
Entities: [('[CLS]', 'O'), ('No', 'O'), ('[SEP]', 'O')]
Text: I'm kidding

In [44]:
import convokit
from convokit import Corpus, Speaker, Utterance

# Initialize a ConvoKit Corpus
corpus = Corpus()

# Example usage
for i, text in enumerate(texts[:10]):  # Analyze first 10 texts
    print("Text:", text)
    entities = bert_ner(text, tokenizer, model)

    # Create a new conversation for each text
    conversation_id = f"conversation_{i}"
    conversation = convokit.Conversation(corpus, conversation_id)

    # Create a user (speaker)
    user = Speaker(corpus, f"user_{i}")

    # Create an utterance
    utterance = Utterance(corpus, user.id, conversation.id, text)

    # Store NER results in the utterance's metadata
    utterance.meta['named_entities'] = entities

    # Add the utterance to the conversation
    conversation.add_utterance(utterance)

    # Add the conversation to the corpus
    corpus.add_conversation(conversation)

# Save the Corpus to a file for later analysis and visualization
corpus.dump("movie-corpus-with-ner")


Text: They do not!


AttributeError: 'str' object has no attribute 'id'